In [2]:
# This code is provided for Deep Learning (CS 482/682) Homework 6 practice.
# The network structure is a simplified U-net. You need to finish the last layers
# @Copyright Cong Gao, the Johns Hopkins University, cgao11@jhu.edu
# Modified by Hongtao Wu on Oct 11, 2019 for Fall 2019 Machine Learning: Deep Learning HW6

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt


# 1 a (first one)

In [3]:
# Functions for adding the convolution layer
def add_conv_stage(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=True, useBN=False):
    if useBN:
        # Use batch normalization
        return nn.Sequential(
          nn.Conv2d(dim_in, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
          nn.BatchNorm2d(dim_out),
          nn.LeakyReLU(0.1),
          nn.Conv2d(dim_out, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
          nn.BatchNorm2d(dim_out),
          nn.LeakyReLU(0.1)
        )
    else:
        # No batch normalization
        return nn.Sequential(
          nn.Conv2d(dim_in, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
          nn.ReLU(),
          nn.Conv2d(dim_out, dim_out, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
          nn.ReLU()
        )


# Upsampling
def upsample(ch_coarse, ch_fine):
    return nn.Sequential(
        nn.ConvTranspose2d(ch_coarse, ch_fine, 4, 2, 1, bias=False),
        nn.ReLU()
        )


# U-Net
class unet(nn.Module):
    def __init__(self, useBN=False):
        super(unet, self).__init__()
        # Downgrade stages
        self.conv1 = add_conv_stage(3, 32, useBN=useBN)
        self.conv2 = add_conv_stage(32, 64, useBN=useBN)
        self.conv3 = add_conv_stage(64, 128, useBN=useBN)
        self.conv4 = add_conv_stage(128, 256, useBN=useBN)
        # Upgrade stages
        self.conv3m = add_conv_stage(256, 128, useBN=useBN)
        self.conv2m = add_conv_stage(128,  64, useBN=useBN)
        self.conv1m = add_conv_stage( 64,  32, useBN=useBN)
        # Maxpool
        self.max_pool = nn.MaxPool2d(2)
        # Upsample layers
        self.upsample43 = upsample(256, 128)
        self.upsample32 = upsample(128,  64)
        self.upsample21 = upsample(64 ,  32)
        # weight initialization
        # You can have your own weight intialization. This is just an example.
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                if m.bias is not None:
                    m.bias.data.zero_()

        #TODO: Design your last layer & activations
#         self.convLast = add_conv_stage(32, 8, useBN=useBN, kernel_size=1, padding=0)
        self.convLast = nn.Conv2d(32, 8, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(self.max_pool(conv1_out))
        conv3_out = self.conv3(self.max_pool(conv2_out))
        conv4_out = self.conv4(self.max_pool(conv3_out))

        conv4m_out_ = torch.cat((self.upsample43(conv4_out), conv3_out), 1)
        conv3m_out  = self.conv3m(conv4m_out_)

        conv3m_out_ = torch.cat((self.upsample32(conv3m_out), conv2_out), 1)
        conv2m_out  = self.conv2m(conv3m_out_)

        conv2m_out_ = torch.cat((self.upsample21(conv2m_out), conv1_out), 1)
        conv1m_out  = self.conv1m(conv2m_out_)

        #TODO: Design your last layer & activations
        convfinal_out = self.convLast(conv1m_out)

        return torch.sigmoid(convfinal_out)

In [ ]:
######################## Hyperparameters #################################
# Batch size can be changed if it does not match your memory, please state your batch step_size
# in your report.
train_batch_size = 10
validation_batch_size=10
# Please use this learning rate for Prob1(a) and Prob1(b)
learning_rate = 0.001
# This num_epochs is designed for running to be long enough, you need to manually stop or design
# your early stopping method.
num_epochs = 1000

# TODO: Design your own dataset
class ImageDataset(Dataset):
    def __init__(self, input_dir):
        datalist = []
        labellist = []
        onehotlabellist = []
        for folders in os.listdir(input_dir):
            new_dir = os.path.join(input_dir, folders)
            if os.path.isdir(new_dir):
                impath = new_dir+ '/'+folders+'_input.jpg'
                labelpath = new_dir+ '/'+folders+'_mask.png'
                im = Image.open(impath)
                imLab = Image.open(labelpath)
                datalist.append(np.array(im))
                labellist.append(np.array(imLab))
        test = np.transpose(np.array(datalist), (0, 3, 1, 2))
        self.data = torch.from_numpy(np.transpose(np.array(datalist), (0, 3, 1, 2))).type('torch.FloatTensor')
        npLabel = np.array(labellist)
        labels = [0, 32, 64, 96, 128, 160, 192, 224]
        print("start process")
        for i in npLabel:
            imgs = []
            for j in labels:
                mask = self.mask(i, j)
                imgs.append(mask)
            onehotlabellist.append(np.array(imgs))
        print("finish process")
        self.label = torch.from_numpy(np.array(onehotlabellist)).type('torch.FloatTensor')
        
        
    def mask(self, label, val):
        return label & val == val
        
    def __len__ (self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
    
    def getData(self):
        return self.data, self.label


# TODO: Implement DICE loss
class DICELoss(nn.Module):
#     def __init__(self, input, target):
#         self.forward(input, target)
    def forward(self, input, target):
        smooth = 1.
        iflat = input.view(-1)
        tflat = target.view(-1)
        intersection = (iflat * tflat).sum()
        dice_coeff = ((2. * intersection + smooth) /
                    (iflat.sum() + tflat.sum() + smooth))
        print("Dice Score: ", dice_coeff)
        return 1 - dice_coeff


# TODO: Use your designed dataset for dataloading
train_dataset=ImageDataset(input_dir = "./HW6_data/segmentation/train/")
validation_dataset=ImageDataset(input_dir =  "./HW6_data/segmentation/validation/")
model = unet(useBN = True)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                       batch_size=train_batch_size, 
                                       shuffle=True)
    
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                           batch_size=validation_batch_size, 
                                           shuffle=True)

training_loss = []
validation_loss = []

print("Start Training...")
for epoch in range(num_epochs):

    ########################### Training #####################################
    print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
    # TODO: Design your own training section
    model.train()
    torch.set_grad_enabled(True)
    for i, data in enumerate(train_loader):
        x = autograd.Variable(data[0]).cuda()
        y = autograd.Variable(data[1]).long().cuda()
        
        optimizer.zero_grad()
        y_hat = model(x)
        train_loss = DICELoss()(y_hat, y)
        train_loss.backward()
        optimizer.step()
    print(train_loss.data)
    training_loss.append(train_loss.data)
    

    ########################### Validation #####################################
    # TODO: Design your own validation section
    print("\nStart Validation...")
    torch.set_grad_enabled(False)
    for i, data in enumerate(validation_loader):
        x = autograd.Variable(data[0]).cuda()
        y = autograd.Variable(data[1]).long().cuda()
        y_hat = model(x)
        val_loss = DICELoss()(y_hat, y)
    print(val_loss.data)
    validation_loss.append(val_loss.data)

# Plot training loss, validation loss
def plot(training_loss, validation_loss):
    plt.figure(300)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')   
    plt.plot(training_loss, 'b')
    plt.figure(400)
    plt.title('Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')   
    plt.plot(validation_loss, 'b')
    plt.show()

start process
finish process
start process
finish process
Start Training...

EPOCH 1 of 1000

Dice Score:  tensor(0.2803, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.2648, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3053, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3065, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3256, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.2999, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3175, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3388, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3230, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3326, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3354, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3388, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.3490, device='cuda:0', grad_fn=<DivBackward0>)
Dice Scor

Dice Score:  tensor(0.6484, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6294, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6330, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6332, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6393, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6640, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6472, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6540, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6591, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6694, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6713, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6496, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6808, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6753, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.6607, device

Dice Score:  tensor(0.8406, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8569, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8472, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.1528, device='cuda:0')

Start Validation...
Dice Score:  tensor(0.8672, device='cuda:0')
Dice Score:  tensor(0.8190, device='cuda:0')
Dice Score:  tensor(0.8436, device='cuda:0')
Dice Score:  tensor(0.8622, device='cuda:0')
Dice Score:  tensor(0.8715, device='cuda:0')
tensor(0.1285, device='cuda:0')

EPOCH 8 of 1000

Dice Score:  tensor(0.8457, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8519, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8640, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8644, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8630, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8230, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8523, device='cuda:0', grad_f

Dice Score:  tensor(0.8957, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9074, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9101, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8916, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8865, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8885, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9034, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9083, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8902, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9158, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9173, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9126, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9157, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8906, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8988, device

Dice Score:  tensor(0.9315, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9111, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9199, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8996, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9210, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9218, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9401, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9239, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8966, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.8949, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.1051, device='cuda:0')

Start Validation...
Dice Score:  tensor(0.9153, device='cuda:0')
Dice Score:  tensor(0.9163, device='cuda:0')
Dice Score:  tensor(0.9034, device='cuda:0')
Dice Score:  tensor(0.9098, device='cuda:0')
Dice Score:  tensor(0.9263, device='cuda:0')
tensor(0.0737, device='cuda:0')


...

Dice Score:  tensor(0.9718, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9706, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9784, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9701, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9817, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9796, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9745, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9646, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9839, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9676, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9797, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9747, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9728, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9668, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9760, device

Dice Score:  tensor(0.9645, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9570, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9587, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9782, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9789, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9830, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9779, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9683, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9731, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9703, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.0297, device='cuda:0')

Start Validation...
Dice Score:  tensor(0.9756, device='cuda:0')
Dice Score:  tensor(0.9399, device='cuda:0')
Dice Score:  tensor(0.9643, device='cuda:0')
Dice Score:  tensor(0.9630, device='cuda:0')
Dice Score:  tensor(0.9452, device='cuda:0')
tensor(0.0548, device='cuda:0')


tensor(0.0263, device='cuda:0')

EPOCH 69 of 1000

Dice Score:  tensor(0.9783, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9796, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9714, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9781, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9573, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9654, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9840, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9842, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9777, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9656, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9822, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9852, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9622, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9806, device='cuda:0', grad_fn

Dice Score:  tensor(0.9504, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9694, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9755, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9805, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9553, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9678, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9817, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9754, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9814, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9640, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9707, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9739, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9781, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9858, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9826, device

Dice Score:  tensor(0.9749, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9636, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9785, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.0215, device='cuda:0')

Start Validation...
Dice Score:  tensor(0.9707, device='cuda:0')
Dice Score:  tensor(0.9709, device='cuda:0')
Dice Score:  tensor(0.9733, device='cuda:0')
Dice Score:  tensor(0.9469, device='cuda:0')
Dice Score:  tensor(0.9616, device='cuda:0')
tensor(0.0384, device='cuda:0')

EPOCH 76 of 1000

Dice Score:  tensor(0.9744, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9786, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9586, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9803, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9838, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9823, device='cuda:0', grad_fn=<DivBackward0>)
Dice Score:  tensor(0.9797, device='cuda:0', grad_

In [ ]:
plot(training_loss, validation_loss)